In [ ]:
import sys
sys.path.append('/tf/data')

import os
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tqdm import tqdm

from general_func import load_dataset

In [ ]:
#load dataset
positive = False
savepath = "/tf/data/augmented_64_3ch/"
new_height = 64
new_width = 64

ds = load_dataset(positive=positive)
if positive == False:
    savepath = savepath+'0'
else:
    savepath = savepath+'1'
os.makedirs(savepath, exist_ok=True)
print(savepath)

In [ ]:
orig_hight = 162
orig_width = 141
num_slices = 46
train_scans = list(np.array([x[1] for x,_ in ds]))
train_scans = (np.reshape(train_scans,[len(ds), num_slices, orig_hight, orig_width, 1]))
print(1,train_scans.shape)
#Perform data augmentations
#Linearly scales each image in image to have mean 0 and variance 1.
train_scans = [tf.image.per_image_standardization(scan) for scan in train_scans] #Linearly scales each image in image to have mean 0 and variance 1.

print('original', len(train_scans))
count = 0
for idx, scan in enumerate(tqdm(train_scans, desc='Augmenting')):
    i = 0
    for angle in [-0.05, -0.025, 0, 0.025, 0.05]:
        scan_a = tfa.image.rotate(images = scan, angles = angle, fill_mode='nearest')
        for contrast in [0.8, 0.9, 1.1, 1.2]:
            scan_b = tf.image.adjust_contrast(scan_a, contrast_factor = contrast)
            for brightness in [-0.1, 0.1]:
                scan_c = tf.image.adjust_brightness(scan_b, delta=brightness) #could have used tf.image.random_brightness but prefer to use fixes values for reproducibility.
                for flip in [True, False]:
                    i += 1
                    if flip == True:
                        scan_d = tf.image.flip_left_right(scan_c)
                    else:
                        scan_d = scan_c
                    scan_d = tf.image.resize(scan_d, [new_height, new_width])
                    scan_d = tf.image.grayscale_to_rgb(scan_d)
                    scan_d1 = scan_d[::2]
                    scan_np = scan_d1.numpy()
                    scan_np = np.concatenate((scan_np), axis = 1)
                    np.save(file = f"{savepath}/image_{idx:04d}_{i:03d}", arr = scan_np)

                    i += 1
                    scan_d2 = scan_d[1::2]
                    scan_np = scan_d2.numpy()
                    scan_np = np.concatenate((scan_np), axis = 1)
                    np.save(file = f"{savepath}/image_{idx:04d}_{i:03d}", arr = scan_np)
    count += i
print('Final format:', scan_np.shape)
print(count,'files saved to', savepath)